In [4]:
import pandas as pd

In [5]:
df_titanic = pd.read_csv("./titanic1.csv")
df_titanic.isna().head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,False,False,False,False,False,False,False,False,False,False,True,False
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,True,False
3,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,True,False


### Podemos usar el método -sum() para obtener el número de nulos por xolumna (True -> 1 y False -> 0)

In [6]:
df_titanic.isna().sum() # El .isna().sum() nos da las pistas de cuantos NaN hay

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

### Número total de NaN

In [9]:
df_titanic.isna().sum().sum()

866

### Eliminación de valores 

In [10]:
df_titanic.shape

(891, 12)

In [11]:
# Eliminamos filas que tengan algún nulo
t = df_titanic.dropna(axis = 0, how = "any")
t.shape

(183, 12)

Esto que hemos hecho ahí arriba es MUY PELIGROSO, ya que si hacemos eso nos podemos quedar con la mitad de los datos

Mejor **NO HACERLO NUNCA**

In [14]:
t2 = df_titanic.dropna(axis = 1, thresh = 700)
t2.shape

(891, 11)

In [15]:
t2

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C


### Se ha eliminado una columna, vemos que "Cabin" tiene menos de 700 valores NO NULOS (si tenemos más de 700 bueno (no NaN) me los quedo)

In [16]:
df_titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


### Imputar un valor condicional

In [17]:
# Vamos a buscar la edad media por géneros (sexo) pra despues sustituir los NaN de mujeres o hombres
EdadMedia_sex = df_titanic.groupby("Sex").mean(numeric_only=True)["Age"] # numeric_only = True no hace caso a los NaN
EdadMedia_sex

Sex
female    27.915709
male      30.726645
Name: Age, dtype: float64

### Hacemos que el  índice coindida conl el de la característicos categçorica que va a detreminad el valor a imutar a cada nulo.

In [18]:
df_titanic.set_index("Sex")["Age"]

Sex
male      22.0
female    38.0
female    26.0
female    35.0
male      35.0
          ... 
male      27.0
female    19.0
female     NaN
male      26.0
male      32.0
Name: Age, Length: 891, dtype: float64

In [19]:
t = df_titanic.set_index("Sex")["Age"].fillna(EdadMedia_sex)
t

Sex
male      22.000000
female    38.000000
female    26.000000
female    35.000000
male      35.000000
            ...    
male      27.000000
female    19.000000
female    27.915709
male      26.000000
male      32.000000
Name: Age, Length: 891, dtype: float64

In [ ]:
### Podríamos plantearnos tammbién ...

In [20]:
EdadMedia_sex_class = df_titanic.groupby(["Sex", "Pclass"]).mean(numeric_only = True)["Age"]
EdadMedia_sex_class

Sex     Pclass
female  1         34.611765
        2         28.722973
        3         21.750000
male    1         41.281386
        2         30.740707
        3         26.507589
Name: Age, dtype: float64

In [21]:
# Cogemos como indice Sexo y Pclass
df_titanic.set_index(["Sex", "Pclass"])["Age"]

Sex     Pclass
male    3         22.0
female  1         38.0
        3         26.0
        1         35.0
male    3         35.0
                  ... 
        2         27.0
female  1         19.0
        3          NaN
male    1         26.0
        3         32.0
Name: Age, Length: 891, dtype: float64

In [23]:
t2 = df_titanic.set_index(["Sex", "Pclass"])["Age"].fillna(EdadMedia_sex_class)
t2

Sex     Pclass
male    3         22.00
female  1         38.00
        3         26.00
        1         35.00
male    3         35.00
                  ...  
        2         27.00
female  1         19.00
        3         21.75
male    1         26.00
        3         32.00
Name: Age, Length: 891, dtype: float64